In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
from resonator_builder import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
rb=ResonatorBuilder()
rb